In [189]:
from pathlib import Path
import os
import pandas as pd
ROOT = 0
ROOT = Path.cwd().parents[3]
os.chdir(ROOT)
df = pd.read_csv(ROOT/"data"/"processed"/"gdelt_ohlcv_join.csv")
ROOT

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\data\\processed\\gdelt_ohlcv_join.csv'

In [12]:
df.head()

,seendate,url,title,language,domain,socialimage,company,ticker,sentiment_score,sentiment_hits,sentiment_present,article_date,price_date,next_open,next_high,next_low,next_close,next_adj_close,next_volume
0,2026-01-11 00:45:00+00:00,https://finance.yahoo.com/news/viasat-stock-bu...,Is Viasat Stock a Buy or Sell After the CEO So...,English,finance.yahoo.com,https://s.yimg.com/ny/api/res/1.2/ApxtKH_0tOj5...,Apple,AAPL,0.96,1.0,True,2026-01-11,2026-01-12,259.160004,261.299988,256.799988,260.25,260.006714,45263800
1,2026-01-11 01:15:00+00:00,https://finance.yahoo.com/news/why-warren-buff...,Why Warren Buffett Finally Bought Alphabet Shares,English,finance.yahoo.com,https://s.yimg.com/os/en/motleyfool.com/4716d7...,Apple,AAPL,0.96,1.0,True,2026-01-11,2026-01-12,259.160004,261.299988,256.799988,260.25,260.006714,45263800
2,2026-01-11 01:15:00+00:00,https://myhostnews.com/iphone-17e-iphone-18-pr...,"iPhone 17e , iPhone 18 Pro , iOS 27 , iPhone F...",English,myhostnews.com,https://myhostnews.com/wp-content/uploads/2026...,Apple,AAPL,0.76,1.0,True,2026-01-11,2026-01-12,259.160004,261.299988,256.799988,260.25,260.006714,45263800
3,2026-01-11 01:15:00+00:00,https://finance.yahoo.com/news/applied-optoele...,Is Applied Optoelectronics Stock a Buy or Sell...,English,finance.yahoo.com,https://s.yimg.com/ny/api/res/1.2/HkyrHDK309aB...,Apple,AAPL,0.96,1.0,True,2026-01-11,2026-01-12,259.160004,261.299988,256.799988,260.25,260.006714,45263800
4,2026-01-11 01:45:00+00:00,https://economictimes.indiatimes.com/markets/s...,Large - cap stocks : Buy right in 2026 & sit q...,English,economictimes.indiatimes.com,"https://img.etimg.com/thumb/msid-126452256,wid...",Apple,AAPL,0.91,4.0,True,2026-01-11,2026-01-12,259.160004,261.299988,256.799988,260.25,260.006714,45263800


In [15]:
print(df.columns)
print(df.shape)

Index(['seendate', 'url', 'title', 'language', 'domain', 'socialimage',
       'company', 'ticker', 'sentiment_score', 'sentiment_hits',
       'sentiment_present', 'article_date', 'price_date', 'next_open',
       'next_high', 'next_low', 'next_close', 'next_adj_close', 'next_volume'],
      dtype='str')
(1309, 19)


In [22]:
tickers = df["ticker"].unique()
print(tickers)

<StringArray>
['AAPL', 'AMZN', 'GOOGL', 'META', 'MSFT', 'NVDA', 'TSLA']
Length: 7, dtype: str


In [5]:
g_ss = df.groupby("ticker")["sentiment_score"]
sent_rg = (g_ss.max() - g_ss.min()).rename("sentiment_range")
print(sent_rg)

ticker
AAPL     1.92
AMZN     1.96
GOOGL    1.92
META     1.92
MSFT     1.92
NVDA     1.92
TSLA     1.92
Name: sentiment_range, dtype: float64


In [6]:
sent_std = g_ss.std().rename("sentiment_std")
sent_std

ticker
AAPL     0.508578
AMZN     0.479773
GOOGL    0.480502
META     0.510481
MSFT     0.439929
NVDA     0.520915
TSLA     0.519599
Name: sentiment_std, dtype: float64

In [21]:
g_t_ad = df.groupby(["ticker","article_date"])["sentiment_score"]
diff = g_t_ad.agg(
    positive = lambda x: (x >= 0).sum(),
    negative = lambda x: (x < 0).sum()
)
diff["net"] = diff["positive"] - diff["negative"]
diff

positive  negative  net
ticker article_date                         
AAPL   2026-01-11          23         3   20
       2026-01-12          40         3   37
       2026-01-27          11         1   10
       2026-02-03          13         0   13
       2026-02-09          31         4   27
AMZN   2026-01-11          12         2   10
       2026-01-12          27         0   27
       2026-01-13          16         3   13
       2026-01-27           9         0    9
       2026-02-03           5         0    5
       2026-02-04           3         0    3
       2026-02-09          30         5   25
GOOGL  2026-01-11           9         0    9
       2026-01-12          18         2   16
       2026-01-13          28         1   27
       2026-01-27          13         0   13
       2026-02-03          13         6    7
       2026-02-04           6         1    5
       2026-02-07          27         4   23
       2026-02-08          40         3   37
       2026-02-09          68         7   61
META   2026-01-11          15         0   15
       2026-01-12          23         6   17
       2026-01-13          62         6   56
       2026-01-14          28         2   26
       2026-01-27           9         0    9
       2026-02-03           9         0    9
       2026-02-04           3         0    3
       2026-02-06          57        19   38
       2026-02-07          39         9   30
       2026-02-08          24         6   18
       2026-02-09          32         4   28
MSFT   2026-01-11          16         0   16
       2026-01-12          23         4   19
       2026-01-13          23         0   23
       2026-01-27          14         2   12
       2026-02-03          14         0   14
       2026-02-09          38         2   36
NVDA   2026-01-11          44         0   44
       2026-01-12          50         2   48
       2026-01-13          22         1   21
       2026-01-27          22         1   21
       2026-02-03           8         1    7
       2026-02-04           2         0    2
       2026-02-09          68        12   56
TSLA   2026-01-11          13         5    8
       2026-01-12          16         2   14
       2026-01-13          13         0   13
       2026-01-27          14         1   13
       2026-02-03           3         1    2
       2026-02-07           6         1    5
       2026-02-08          14         1   13
       2026-02-09           8         2    6

In [67]:
qs = (
    diff.groupby("ticker")["net"]
    .quantile([0.25, 0.75])
    .unstack()
    .rename(columns={0.25: "q1", 0.75: "q3"})
)
qs

,q1,q3
ticker,,
AAPL,13.00,27.0
AMZN,7.00,19.0
GOOGL,9.00,27.0
META,12.00,29.0
MSFT,14.50,22.0
NVDA,14.00,46.0
TSLA,5.75,13.0


In [68]:
qs["iqr"] = qs["q3"] - qs["q1"]
qs["lower"] = qs["q1"] - (1.5 * qs["iqr"])
qs["upper"] = qs["q3"] + (1.5 * qs["iqr"])
qs

,q1,q3,iqr,lower,upper
ticker,,,,,
AAPL,13.00,27.0,14.00,-8.000,48.000
AMZN,7.00,19.0,12.00,-11.000,37.000
GOOGL,9.00,27.0,18.00,-18.000,54.000
META,12.00,29.0,17.00,-13.500,54.500
MSFT,14.50,22.0,7.50,3.250,33.250
NVDA,14.00,46.0,32.00,-34.000,94.000
TSLA,5.75,13.0,7.25,-5.125,23.875


msft is the only one with a positive lower range and second smallest iqr

In [180]:
diff2 = diff.join(qs[["lower", "upper"]], on="ticker")
xtrm_diff = diff2[ (diff2["net"]<=diff2["lower"]) | (diff2["net"]>=diff2["upper"]) ].reset_index()
xtrm_diff

,ticker,article_date,positive,negative,net,lower,upper
0,GOOGL,2026-02-09,68,7,61,-18.00,54.00
1,META,2026-01-13,62,6,56,-13.50,54.50
2,MSFT,2026-02-09,38,2,36,3.25,33.25


googl and msft on the same day?

In [123]:
xdf=df.merge(xtrm_diff[["article_date","ticker"]], how="inner", on=["article_date","ticker"], )

In [177]:
xdomains=xdf.groupby(["article_date","domain"]).size()
xdomains=(xdomains[xdomains>6].reset_index(name="count")
     )
xdomains

,article_date,domain,count
0,2026-01-13,markets.financialcontent.com,8
1,2026-02-09,finance.yahoo.com,11
2,2026-02-09,fool.com,16
3,2026-02-09,markets.financialcontent.com,41


In [188]:
xdf[xdf["article_date"]=="2026-01-13"]

,seendate,url,title,language,domain,socialimage,company,ticker,sentiment_score,sentiment_hits,sentiment_present,article_date,price_date,next_open,next_high,next_low,next_close,next_adj_close,next_volume
0,2026-01-13 00:00:00+00:00,https://www.pymnts.com/meta/2026/meta-picks-fo...,Meta Picks Former White House Advisor to Drive...,English,pymnts.com,https://www.pymnts.com/wp-content/uploads/2025...,Meta Platforms,META,0.00,0.0,False,2026-01-13,2026-01-14,626.5,628.450012,614.820007,615.52002,615.52002,15527900
1,2026-01-13 01:00:00+00:00,https://www.bostonglobe.com/2026/01/12/busines...,Malaysia and Indonesia become the first countr...,English,bostonglobe.com,https://bostonglobe-prod.cdn.arcpublishing.com...,Meta Platforms,META,0.00,0.0,False,2026-01-13,2026-01-14,626.5,628.450012,614.820007,615.52002,615.52002,15527900
2,2026-01-13 01:00:00+00:00,https://www.webpronews.com/meta-pivots-to-prop...,Meta Pivots to Proprietary AI in 2026 with Nuc...,English,webpronews.com,https://www.webpronews.com/wp-content/uploads/...,Meta Platforms,META,0.00,0.0,False,2026-01-13,2026-01-14,626.5,628.450012,614.820007,615.52002,615.52002,15527900
3,2026-01-13 01:00:00+00:00,https://www.webpronews.com/china-probes-metas-...,China Probes Meta $2B AI Startup Buy Amid Expo...,English,webpronews.com,https://www.webpronews.com/wp-content/uploads/...,Meta Platforms,META,0.64,2.0,True,2026-01-13,2026-01-14,626.5,628.450012,614.820007,615.52002,615.52002,15527900
4,2026-01-13 01:45:00+00:00,https://www.siliconvalley.com/2026/01/12/meta-...,Meta plans to cut 10 % of jobs at company real...,English,siliconvalley.com,NaN,Meta Platforms,META,0.00,0.0,False,2026-01-13,2026-01-14,626.5,628.450012,614.820007,615.52002,615.52002,15527900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,2026-01-13 23:00:00+00:00,https://markets.financialcontent.com/stocks/ar...,FinancialContent - Micron Metamorphosis : How ...,English,markets.financialcontent.com,https://marketminute.ghost.io/content/images/s...,Meta Platforms,META,0.00,0.0,False,2026-01-13,2026-01-14,626.5,628.450012,614.820007,615.52002,615.52002,15527900
64,2026-01-13 23:00:00+00:00,https://markets.financialcontent.com/stocks/ar...,FinancialContent - The Great Rotation : Small ...,English,markets.financialcontent.com,https://marketminute.ghost.io/content/images/s...,Meta Platforms,META,0.96,1.0,True,2026-01-13,2026-01-14,626.5,628.450012,614.820007,615.52002,615.52002,15527900
65,2026-01-13 23:15:00+00:00,https://www.webpronews.com/meta-lays-off-10-of...,"Meta Lays Off 10 % of Reality Labs Staff , Clo...",English,webpronews.com,https://www.webpronews.com/wp-content/uploads/...,Meta Platforms,META,0.00,0.0,False,2026-01-13,2026-01-14,626.5,628.450012,614.820007,615.52002,615.52002,15527900
66,2026-01-13 23:15:00+00:00,https://www.webpronews.com/ignitetech-ceo-cuts...,"IgniteTech CEO Cuts 80 % Staff for AI Shift , ...",English,webpronews.com,https://www.webpronews.com/wp-content/uploads/...,Meta Platforms,META,0.00,0.0,False,2026-01-13,2026-01-14,626.5,628.450012,614.820007,615.52002,615.52002,15527900


In [186]:
nx_df=df.merge(
    xtrm_diff[["article_date","ticker"]]
    , how="left"
    , on=["article_date","ticker"]
    ,indicator=True
)
nx_df = nx_df[nx_df["_merge"] == "left_only"]
nx_df[(nx_df["ticker"]=="MSFT") & (nx_df["article_date"]=="2026-02-09")].head()

,seendate,url,title,language,domain,socialimage,company,ticker,sentiment_score,sentiment_hits,sentiment_present,article_date,price_date,next_open,next_high,next_low,next_close,next_adj_close,next_volume,_merge


In [187]:
domains=(
    nx_df.groupby(["article_date","domain"]).size()
    .reset_index(name="count")
)
filtered_d=domains.merge(xdomains["domain"], how = "inner", on="domain")
filtered_d.groupby("domain")["count"].mean()

domain
finance.yahoo.com               18.363636
fool.com                         9.400000
markets.financialcontent.com    12.428571
Name: count, dtype: float64